In [1]:
from nd2reader import ND2Reader
import tifffile as tif
import numpy as np
import os.path as osp
import imageio
import tqdm
import glob

In [2]:
noise_path = 'noise_111125.tif'
noise_tif = tif.imread(noise_path)
noise_stack = np.stack([noise_tif] * 41, axis=0).astype(np.float32)

In [4]:
nd2_path = '/Volumes/SCRATCH3/date-20260114_strain-ISg5HT_condition-starvedpatch_worm001.nd2'
stack_shape = (41, 2, 512, 512)
fps = 5 / 0.533

def get_stack(f, index, trim=1):
    stack = np.zeros(stack_shape, np.float32)
    frame_indices = np.arange(stack_shape[0] * index, stack_shape[0] * (index + 1))
    for i, j in enumerate(frame_indices):
        stack[i] = f.get_frame_2D(0, j), f.get_frame_2D(1, j)
    denoised = np.clip(stack - noise_stack, 0, 4095)
    return denoised[trim:]

def make_rgb(stack):
    gfp, rfp = np.max(stack, axis=0)
    rgb = np.zeros((*rfp.shape, 3), np.uint8)
    rgb[..., 0] = np.clip(rfp / 500, 0, 1) * 255
    rgb[..., 1] = np.clip(gfp / 200, 0, 1) * 255
    return rgb

with ND2Reader(nd2_path) as f:
    stack_range = range(f.metadata['num_frames'] // stack_shape[0])
    mp4_path = osp.splitext(nd2_path)[0] + '.mp4'
    if osp.exists(mp4_path):
        pass

    with imageio.get_writer(mp4_path, fps=fps) as mp4:
        for i in tqdm.tqdm(stack_range):
            stack = get_stack(f, i)
            rgb = make_rgb(stack)
            mp4.append_data(rgb)

100%|██████████| 1200/1200 [04:05<00:00,  4.89it/s]


In [ ]:
nd2_paths = glob.glob('/Volumes/SCRATCH3/*.nd2')

for nd2_path in nd2_paths:
    mp4_path = osp.splitext(nd2_path)[0] + '_quick.mp4'
    if osp.exists(mp4_path):
        continue

    with ND2Reader(nd2_path) as f:
        stack_range = range(f.metadata['num_frames'] // stack_shape[0])
        with imageio.get_writer(mp4_path, fps=fps) as mp4:
            for i in tqdm.tqdm(stack_range, leave=False):
                stack = get_stack(f, i)
                mp4.append_data(make_rgb(stack))